<h1 style="text-align:center; font-size:35.0px">BI Analysis of Traffic Sources</h1>

<img src="skyscraper_sd.png" alt="analytics SD art">
<span style='font-family:Sabon;font-size:8.0pt;'>
    <b>Image generated via <a href='https://github.com/CompVis/stable-diffusion'>Stable Diffusion</a></b>
</span>

<span style='font-family:Sabon;font-size:24.0pt;'>
    <b>Table of Contents</b>
</span>
<span style='font-family:Sabon;font-size:16.0pt;line-height:1.5'>
    <ul>        
        <li><a href='#intro'>1. Introduction</a></li>
        <li><a href='#func'>2. Define Functions</a></li>
        <li><a href='#import_data'>3. Import Data and Libraries</a></li>
        <li><a href='#gen_info'>4. General Information</a></li>        
        <li><a href='#pp'>5. Preparing Data for Analysis</a></li>
        <ul>
            <li><a href='#pp_dup'>5.1. Check for Duplicates</a></li>
            <li><a href='#pp_miss'>5.2. Checking for Missing Values</a></li>
            <li><a href='#pp_ts'>5.3. Date Distribution</a></li>
            <li><a href='#pp_dur'>5.4. Calculating Session Duration</a></li>
            <li><a href='#pp_sql'>5.5. Merging Visits and Orders Using PandaSQL</a></li>
            <li><a href='#pp_dres'>5.6. Date Restriction</a></li>
            <li><a href='#pp_dist'>5.7. Revenue Distribution</a></li>
            <li><a href='#pp_dev'>5.8. Device Usage and Distribution</a></li>
        </ul>
        <li><a href='#stat_sum'>6. Traffic Source Overall Performance</a></li>
        <ul>
            <li><a href='#stat_dev_1'>6.1 Device Usage Across Sources</a></li>
            <li><a href='#stat_cost'>6.2. Average Monthly Costs by Source</a></li>
            <li><a href='#gen_sum'>6.3. General Statistical Summary by Source</a></li>
        </ul>
        <li><a href='#marketing'>7. Marketing Success Analysis</a></li>
            <ul>
                <li><a href='#marketing_attr'>7.1. Attribution</a></li>
                <li><a href='#marketing_cohort'>7.2. Cohort Prepping</a></li>
                <ul>
                    <li><a href='#marketing_cohort_res'>7.2.1. Cohort Initial Results</a></li>
                </ul>
                <li><a href='#marketing_romi'>7.3. Calculating Return On Marketing Investment</a></li>
                <li><a href='#marketing_acq'>7.4. Acquisition Pull</a></li>
                <li><a href='#marketing_sum'>7.5. Marketing Success Summary Visualization</a></li>
                <ul>
                    <li><a href='#marketing_s1'>7.5.1. Analysis of Source 1</a></li>
                    <li><a href='#marketing_s3'>7.5.2. Analysis of Source 3</a></li>
                    <li><a href='#marketing_s4'>7.5.3. Analysis of Source 4</a></li>
                    <li><a href='#marketing_s5'>7.5.4. Analysis of Source 5</a></li>
                    <li><a href='#marketing_s10'>7.5.5. Analysis of Source 10</a></li>
                </ul>
                <li><a href='#marketing_reppurchases'>7.6. Channel Purchase Redirection</a></li>
            </ul>    
        <li><a href='#conc_rec'>Conclusions and Recommendations</a></li>
    </ul>
</span>

<a id="intro"></a>
<h2>1. Introduction</h2>

In this project we work with data from an E-Commerce platform.<br>
We have access to datasets consisting of records of user visits to the service, purchases, and marketing costs for several of the many traffic channels the company employs to attract users and direct them to the service.<br><br>
The purpose of this analysis is to determine which of the traffic channels we are given to consider, are working for the benefit of the company and which are a dead weight.<br>
For this determination we use the following guidelines of success:<br>
<ul>
  <li>Profitability - Return on the marketing investment.</li>
  <li>New user acquisition potential - How many new users start using the service thanks to this source?</li>
  <li>Monthly Retention rate - How stable is this source? Are we retaining users at a high enough rate every month?</li>
  <li>Conversion of users to buyers - Are some traffic channels better at directing users to perform purchases?</li>
  <li>profitability vs user acquisition - Are some channels profitable AND bring in many users? Are some unprofitable but have a big potential to attract new users?</li>
</ul>
We will explore these concepts thoroughly in this analysis using Cohorts, Visualizations, and various KPI's.<br>
Once we have a good idea of what each source is good at, we will summarize the effectiveness of each channel to decide which one is worth keeping and investing in, and which ones are better off discontinued.<br><br>
<b>So let's dive in!</b>


<a id="func"></a>
<h2>2. Define Functions</h2>

In [ ]:
def val_counts(df_list, drop_na= True, n=5):
    
    """
    This function returns a dataframe's 
    column values and their value counts
    in absolute and relative values

    Parameters:
        df_list(list): a list of one or more dataframes.
                       Columns should be specified for
                       the columns which value counts you need.
        n (int): number of rows to report for each column. (Default = 5)
    Returns:
        A report of the first highest n value counts(absolute and relative) in each specified column
        for each df in the df_list.

    """
    #enumerate df list for name reference
    df_num= [i+1 for i, df in enumerate(df_list)]
    
    for df in df_list:
        try:
            df_name = df.custom_name
        except:
            df_name = 'Dataframe {}'.format(df_num[0])
            
        #pop first df number after successful naming
        df_num.pop(0)
        
        print('Top {} Value Counts for {} :\n'.format(n,df_name))
        for column in df.columns:
            print('\nColumn: {}'.format(column))
            val_normalized = df[column].value_counts(normalize=True, dropna= drop_na).to_frame()
            val_absolute = df[column].value_counts(dropna= drop_na).to_frame()
            val_final = val_absolute.merge(val_normalized, right_index=True, left_index=True)
            val_final.columns = ['{}'.format(column+' '+'absolute'),'{}'.format(column+' '+'relative')]
            display(val_final.head(n))
        print('\n___________________________________________________\n')
            
def sample_display(df_list,n=5, random_state= None):
    """
    This function displays samples
    of all the dataframes in the list.
    n rows default 5
        
    Parameters:
    
        df_list: list of pandas dataframes
    
        n (int): number of sample rows to return
        
    Returns:
    
        n sample rows from each df listed
    
    """
    df_num= [i+1 for i, df in enumerate(df_list)]
    for df in df_list:   
        try:
            df_name = df.custom_name
        except:
            df_name = 'Dataframe {}'.format(df_num[0])
    #set df name
        print('{} sample:'.format(df.custom_name))
        display(df.sample(n, random_state=random_state))
        print('\n___________________________________\n')
        
def dup_display(df_list):
    """
    This function returns the
    number of duplicated rows
    in each df in the list
        
    Parameters:
        df_list: list of pandas dataframes
    
    Returns:
        report of duplicated rows in each df listed
        report of duplicates diversity in each df listed
    
    """
    alias= 0
    for df in df_list:
        dup_sum = df.duplicated().sum()
        dup_perc = dup_sum / df.shape[0]
        try:
            df_name = df.custom_name
        except:
            df_name = 'Unknown Dataframe {}'.format(alias)
            alias += 1
        
        print('{} :\nDuplicated Values: {}\n% duplicated rows: {:.2%}\n'.format(
            df_name,dup_sum,dup_perc
        )
             )
        print('Duplicates distribution diversity:')
        for col in df[df.duplicated()].columns:
            print('{} : {}'.format(
                col,df[df.duplicated()][col].nunique()
        )
             )
        print()
    
def percentile_report(series, percentile=[1, 5, 95, 99]):
    """
    Prints a report of the percentiles defined in
    the percentile argument for a given series, as well
    as the minimum and maximum values.
    
    Parameters:
        series(pd.Series): a pandas array
        percentile(list): a list of percentiles to report upon
    Returns:
        A report of values by percentiles.
    """
    print(series.name, 'Percentile Report:\n\n')
    print('Minimum value: {}'.format(series.min()))
    for perc in percentile:
        print('{} Percentile: {}'.format(perc, np.percentile(series, perc)))
        print('___________________________________')
        print('\n')
    print('Maximum value: {}'.format(series.max()))

def cohort_prep(df, uid_col, date_col,col_name_type ,cohort_week= False, cohort_month = False, cohort_year= False):
    """
    This function takes a df with
    an id column and a date column
    and prepares it for cohort analysis.
    naming convention is controlled with
    col_name_type.
    lifetime calculation default is off
    
    Parameters:
        df (Pandas DataFrame): dataframe to transform
        uid_col (str) : name of the user identifier column
        date_col (str) : name of the date column
        col_name_type (str) : this controls naming convention, for example setting this value to 'event'
                              will produce columns such as first_event, event_year, etc.
        cohort_week (bool) :  If True, creates weekly cohort and weekly lifetime columns (Default= False)
        cohort_month (bool) :  If True, creates monthly cohort and monthly lifetime columns (Default= False)
        cohort_year (bool) :  If True, creates yearly cohort and yearly lifetime columns (Default= False)
    
    Returns:
        dataframe with specified cohort columns, ready for aggregation
        
    """

    #produce first date for each user
    first_date = df.groupby(uid_col)[date_col].min()
    first_date.name = 'first_{}'.format(col_name_type)
    
    #merge first dates to original dataframe
    cohort = df.join(first_date, on= uid_col)
    
    
    #produce date columns for calculations
    
    #Yearly Cohorts
    if cohort_year == True:
        #extract year from date
        cohort['{}_year'.format(col_name_type)] = cohort[date_col].dt.year
        
        #produce first year for cohort
        cohort['first_{}_year'.format(col_name_type)] = cohort['first_{}'.format(col_name_type)].dt.year 
        
        #produce cohort lifetime column
        cohort['lifetime_year'] = (
                cohort['{}_year'.format(col_name_type)] - cohort['first_{}_year'.format(col_name_type)]
                ).astype(int)
        
    #monthly Cohorts
    if cohort_month == True:    
        cohort['{}_month'.format(col_name_type)] = cohort[date_col].astype('datetime64[M]')
        
    #produce first month for cohort
        cohort['first_{}_month'.format(col_name_type)] = cohort['first_{}'.format(col_name_type)].astype('datetime64[M]')    
    
    #produce cohort lifetime column
        cohort['lifetime_month'] = (
            ( 
                pd.to_datetime(cohort[date_col]) - pd.to_datetime(cohort['first_{}'.format(col_name_type)])
            ) / np.timedelta64(1,'M')).astype(int)
    
    #Weekly Cohorts
    if cohort_week == True:
        
        #for week calculation we subtract the value of the day of the week from the event date
        cohort['{}_week'.format(col_name_type)] = (
            pd.to_datetime(cohort['{}_date'.format(col_name_type)]) - pd.to_timedelta(pd.to_datetime(cohort['{}_date'.format(col_name_type)]).dt.dayofweek, unit='d')
            )        

        #produce first week for cohort
        cohort['first_{}_week'.format(col_name_type)] = (
            pd.to_datetime(cohort['first_{}'.format(col_name_type)], unit='d') - pd.to_timedelta(cohort['first_{}'.format(col_name_type)].dt.dayofweek, unit='d')
            ).astype('datetime64[D]')

        #produce cohort lifetime column
        cohort['lifetime_week'] = ((
            cohort['{}_week'.format(col_name_type)] - cohort['first_{}_week'.format(col_name_type)]
            ) / np.timedelta64(1,'W')).astype(int)
    
    return(cohort)

<a id="import_data"></a>
<h2>3. Import Data and Libraries</h2>

In [ ]:
!pip install sidetable
!pip install pandasql

In [ ]:
#import Libraries
import random as rd
import pandas as pd
from pandasql import sqldf
import numpy as np
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import sidetable as stb
from matplotlib import pyplot as plt
import seaborn as sns
import warnings

#ignore warnings
warnings.filterwarnings('ignore')

#define pandas sql module to update automatically with available variables
mysql = lambda q: sqldf(q, globals())

In [ ]:
#get current path
path = os.getcwd()

#read data to variable
costs_data = pd.read_csv(
    path + '\\Data\\costs_us.csv',
    parse_dates= ['dt']          
      
)

orders_data = pd.read_csv(
    path + '\\Data\\orders_log_us.csv',
    parse_dates= ['Buy Ts']

)

visits_data = pd.read_csv(
    path + '\\Data\\visits_log_us.csv',
    parse_dates=['End Ts', 'Start Ts']
)

#assign column names and reorder
costs_data.columns = ['source_id', 'date', 'costs']

visits_data.columns = ['device', 'end_ts', 'source_id', 'start_ts', 'user_id']
visits_data = visits_data[['user_id','device', 'source_id', 'start_ts', 'end_ts']]

orders_data.columns = ['purchase_ts', 'revenue', 'user_id']
orders_data = orders_data[['user_id', 'purchase_ts', 'revenue']]

#name dataframe for reference
costs_data.custom_name = 'costs'
orders_data.custom_name = 'orders'
visits_data.custom_name = 'visits'

<a id="gen_info"></a>
<h2>4. General Information</h2>

<span style='font-family:sabon;font-size:13pt;'>
    Let's take a look at our datasets and what kind of information they hold<br><br>
</span>

In [ ]:
#show samples of datasets
sample_display([visits_data, orders_data, costs_data], 2)

In [ ]:
#show revenue preliminary statistics
orders_data.drop('user_id', axis=1).describe().T

In [ ]:
#show costs preliminary statistics
costs_data.drop('source_id', axis=1).describe().T

<span style='font-family:sabon;font-size:13pt;'>
    Our data contains information about user visits and their purchases if any were made.<br>
    We also have the daily marketing costs for each of the traffic channels used for this analysis.
    </span>

<a id='pp'></a>
<h2>5. Preparing Data for Analysis</h2>

In [ ]:
#define df list for preperation
df_list = [costs_data, visits_data, orders_data]

<a id='pp_dup'></a>
<h3>5.1. Checking for Duplicates</h3>

<span style='font-family:sabon;font-size:13pt;'>
    Here we are checking for any duplicated rows within the data.<br>
    As the data contains a unique id column we should not find any duplicates if the data's integrity is preserved.<br>
    As part of our function, duplicate distribution diversity is checked as well, showing us the frequency of repeated values by column for duplicated rows.
    </span>

In [ ]:
#produce duplicate report
dup_display(df_list)

<span style='font-family:sabon;font-size:13pt;'>
    So no duplicates are present!<br>
    Great, but let's see if we detect any missing values in the rows we have.
    </span>

<a id='pp_miss'></a>
<h3>5.2. Checking for Missing Values</h3>

In [ ]:
#produce missing values report using Sidetable
for df in df_list:
    display(df.stb.missing(style=True).background_gradient(subset=['percent'],cmap='Greens'))

<span style='font-family:sabon;font-size:13pt;'>
    Luckily our dataset is fairly clean and has no apparent missing values.
    </span>

<a id='pp_ts'></a>
<h3>5.3. Date Distribution</h3>

In [ ]:
#plot histogram of dates distribution
fig= px.histogram(
    visits_data['start_ts'],
    nbins= 50,
    title= 'Distribution of Session Dates',
    template='seaborn'
)

#remove grid
fig.update_yaxes(
    showgrid=False,
    title='Count'
)
fig.update_xaxes(
    showgrid=False,
    title='Start Date'
)


#change title attributes
fig.update_layout(
    title_font=dict(
        family="Arial Black",
        size=25
        ),
    title_x = 0.5,
    showlegend = False
)
fig.show()

<span style='font-family:sabon;font-size:13pt;'>
    Our data stretches from June 2017 to May 2018 inclusive.
    </span>

<a id='pp_dur'></a>
<h3>5.4. Calculating Session Duration</h3>

In [ ]:
#create session duration column
visits_data['session_duration'] = (visits_data['end_ts'] - visits_data['start_ts']) / np.timedelta64(1, 's')

In [ ]:
#show statistics for session duration
visits_data[['session_duration']].describe()

<span style='font-family:sabon;font-size:13pt;'>
    We see that we have some <b>negative values</b> in the session duration calculation.<br>
    Let's check them out now.
</span>

In [ ]:
#produce rows where session duration is negative
visits_data.query("session_duration < 0")

<span style='font-family:sabon;font-size:13pt;'>
    We have an instance where an end_ts was recorded <b>before</b> a start_ts, causing the session duration to result in a negative value.<br>
Let's take a look at those users average session times and see if it makes sense to fill those values with ones that make sense.
</span>

In [ ]:
#retrieve user ids where session duration is negative
bad_ids = visits_data.query("session_duration < 0")['user_id']
visits_data[visits_data['user_id'].isin(bad_ids)].sort_values('user_id').style.background_gradient(subset=['session_duration'],cmap='Greens')

<span style='font-family:sabon;font-size:13pt;'>
    We see that this user uses multiple sources to access the service.<br>
    If we look at the average session durations for the users in the source they used to access the service, we see that the session duration logged wrongly, is not far off from their average session duration.<br>In this case we will simply reverse the sign and assume that the opposite values was an error in logging.
    </span>

In [ ]:
#transform negative session durations to positive
visits_data['session_duration'] = visits_data['session_duration'].apply(abs)

<span style='font-family:sabon;font-size:13pt;'>
    Let's also switch between the start_ts and end_ts for those rows, so that our analysis going forward will be consistent.
    </span>

In [ ]:
#replace start_ts with end_ts and vice versa for problematic rows
for idx in bad_ids.index:
    print('replacing timestamp for entry:\n')
    display(visits_data.loc[[idx]][['user_id', 'start_ts', 'end_ts']])
    start = visits_data.loc[idx]['start_ts']
    end = visits_data.loc[idx]['end_ts']
    visits_data.loc[[idx], 'start_ts'] = end
    visits_data.loc[[idx], 'end_ts'] = start
    print('updated entry:\n')
    display(visits_data.loc[[idx]][['user_id', 'start_ts', 'end_ts']])

In [ ]:
#show updated statistics for session duration
visits_data[['session_duration']].describe()

<span style='font-family:sabon;font-size:13pt;'>
    We took care of the negative values and yet we still have session with a duration of 0.<br>
    These could still be rather informative, though we may filter them out when constructing our active user criteria.<br>
    For now, let's take a look at the outliers in the upper limits of the session durations, as those seem rather unnaturally high.
    </span>

In [ ]:
#show percentile report for session durations in minutes
percentile_report(visits_data['session_duration']/60, [50, 95, 97, 99])

In [ ]:
#define style for axis headings
subtitle_style = dict(
                family='Arial Black',
                size= 15
            )
#plot box distrbution for session durations
fig = go.Figure(
    go.Box(
        y=visits_data['session_duration']/60,
        marker = dict(
        color='black'
    )
    )
)


fig.update_layout(
    title= dict(
        text= 'IQR Analysis of Session Durations (Minutes)',
        font= dict(
            family = 'Arial Black',
            size= 25
            ),
        x=0.5
    ),
    yaxis_showgrid= False,
    xaxis_title = dict( text='Visits', font=subtitle_style),
    yaxis_title = dict( text='Session Duration', font=subtitle_style)
    
)
fig.show()

<span style='font-family:sabon;font-size:13pt;'>
    We see that up to 95% of the data has a session duration of less than 38 minutes, as opposed to the maximum value of 711 minutes (over 10 hours!).<br>
    With a median of 5 minutes we have to include values which reflect typical user behavior without losing too much information.<br>
    In our case we will remove sessions that are over the 95th percentile.<br>
    Before we remove valuable data, lets check if the sessions to be removed are involved in any purchasing activity we need to pay attention to.<br><br>
    For this we need to merge our data and connect sessions to sales.
    </span>

<a id='pp_sql'></a>
<h3>5.5. Merging Visits and Orders Using PandaSQL</h3>

<span style='font-family:sabon;font-size:13pt;'>
    To join our orders table to our visits table, we must first identify which orders belong to which sessions.<br>
    Since our data is missing session ids, we will rely on user ids and timestamps.<br>
    In our case, we will assign an order to a session if all of the following criteria are true:
    <ul>
        <li>User id is identical</li>
        <li>order timestamp is within the range of the sessions start_ts and end_ts</li>
    </ul>
    In order to apply these rules easily, we will use the pandasql module which enables us to use SQL language to join the tables together based on both conditions simultaneously.<br>
    </span>

In [ ]:
#change dtype to be compatible with pandasql
visits_data.user_id = visits_data.user_id.astype('int64')
visits_data.user_id = visits_data.user_id.apply(abs)

orders_data.user_id = orders_data.user_id.astype('int64')
orders_data.user_id = orders_data.user_id.apply(abs)

In [ ]:
query = """
        SELECT
            vd.user_id,
            vd.device,
            vd.source_id,
            vd.start_ts,
            vd.end_ts,
            od.purchase_ts,
            od.revenue,
            vd.session_duration
        FROM
            visits_data vd LEFT JOIN orders_data od
        ON
            vd.user_id = od.user_id
            AND
                (od.purchase_ts >= vd.start_ts
                AND
                od.purchase_ts <= vd.end_ts)
        """
vo_merge = mysql(query)

In [ ]:
for col in ['start_ts' , 'end_ts' , 'purchase_ts']:
    vo_merge[col] = pd.to_datetime(vo_merge[col])
vo_merge.info()

<a id='pp_dres'></a>
<h3>5.6. Date Restriction</h3>

In [ ]:
#set date restriction to june 2017 to end of may 2018
start_dt = '2017-06-01'
end_dt = '2018-06-01'
start_mask = (vo_merge['start_ts'] >= start_dt) & (vo_merge['start_ts'] < end_dt)
end_mask = (vo_merge['end_ts'] < end_dt) & (vo_merge['end_ts'] >= start_dt)

#filter data
vo_merge = vo_merge[(start_mask) & (end_mask)]

In [ ]:
#show percentile report for session durations in minutes
percentile_report(vo_merge[~vo_merge['purchase_ts'].isna()]['session_duration']/60, [50, 95, 97, 99])

In [ ]:
percentile_report(vo_merge[vo_merge['purchase_ts'].isna()]['session_duration']/60, [50, 95, 97, 99])

<span style='font-family:sabon;font-size:13pt;'>
    We see that sessions with purchases tend to be about twice as long as sessions without purchases.<br>
    if longer sessions are not typical for certain sources we'll simply remove ones over 100 minutes so that we can have a coherent look at our baseline performance.
    </span>

In [ ]:
#aggregate session duration by source id
(vo_merge.groupby('source_id', as_index=True).agg({'session_duration' : ['mean', 'median']})/60)\
    .style.background_gradient(subset=[('session_duration','mean'), ('session_duration','median')],cmap='Greens')

<span style='font-family:sabon;font-size:13pt;'>
    We do see that Source 1 does seem to produce a higher session duration, however this is still within the boundaries of 0-10 minutes by median.<br>
    Therefore we can safely assume that values over 100 minutes are going to be outliers for all sources, and so we will remove those.<br>
    </span>

In [ ]:
#set session limit and filter
session_limit = 100*60
vo_merge = vo_merge.query('session_duration <= @session_limit')

<a id='pp_dist'></a>
<h3>5.7. Revenue Distribution</h3>

In [ ]:
col='revenue'
fig= px.histogram(
    orders_data['revenue'],
    #nbins= 30,
    title= 'Distribution of Revenue',
    template='seaborn'
)

#remove grid
fig.update_yaxes(
    showgrid=False,
    title='Count'
)
fig.update_xaxes(
    showgrid=False,
    title='Revenue'
)


#change title attributes
fig.update_layout(
    font=dict(
        family="Arial Black",
        size=12
        ),
    xaxis_dtick=100,
    title_x = 0.5
)
fig.show()

<span style='font-family:sabon;font-size:13pt;'>
    We see that most revenues are under 100USD, with extreme outliers after the 2000's
    </span>

In [ ]:
vo_merge[~vo_merge['purchase_ts'].isna()]['source_id'].unique()

In [ ]:
#plot IQR analysis of revenue by source
fig= go.Figure(
    go.Box(
        y=vo_merge[~vo_merge['purchase_ts'].isna()]['revenue'],
        x=vo_merge[~vo_merge['purchase_ts'].isna()]['source_id'],
        marker= dict(color='black')
        )
    )

fig.update_layout(
    title= dict(
            text= 'IQR Analysis of Revenue by Source',
            font= dict(
                family= 'Arial Black',
                size= 25
            ),
            x=0.5
    ),
    xaxis=dict(
        showgrid=False,
        type = "category",
        categoryorder = 'array',
        categoryarray=sorted(vo_merge['source_id'].unique())
        ),
    xaxis_title= dict(
            text= 'Source ID',
            font= subtitle_style
            ),
    yaxis_title= dict(
            text= 'Revenue',
            font= subtitle_style
            ),
    yaxis_showgrid= False
)

<span style='font-family:sabon;font-size:13pt;'>
    We see that we have some extreme outliers in revenue.<br>
    In this case we will examine the top percentiles and pick a revenue limit to filter our data with, in order to achieve a baseline idea of the performance of each source.
    </span>

In [ ]:
percentile_report(vo_merge[~vo_merge['purchase_ts'].isna()]['revenue'], [50, 95, 99])

In [ ]:
over_100 = vo_merge[~vo_merge['purchase_ts'].isna()]\
            .query('revenue > 100').shape[0] / vo_merge[~vo_merge['purchase_ts'].isna()].shape[0]
print(f'% of purchases over 100 USD: {over_100:.5%}')

<span style='font-family:sabon;font-size:13pt;'>
    We see that most of our data contains revenue of up to about 26 USD.<br>
    We will raise the revenue bar to 100 USD and filter out the rest.<br>
    We also see that we have revenues that are 0 USD, we will dive into these and see if it is appropriate to remove them.
    </span>

In [ ]:
rev_0 = vo_merge[~vo_merge['purchase_ts'].isna()]\
    .query('revenue == 0').shape[0] / vo_merge[~vo_merge['purchase_ts'].isna()].shape[0]
print(f'% of purchases with revenue 0 USD: {rev_0:.5%}')

<span style='font-family:sabon;font-size:13pt;'>
    We have some purchases which sum up to 0 USD, these could perhaps be promo codes or special discounts given to certain customers, however these make up less than 0.1% of the data and therefore are irrelevant for our case.<br>
    We will remove them in the next step.
    </span>

In [ ]:
#filter out revenues 
#collect indexes of instances where revenue is 0 and those where revenue is over 100
zrev_idx = np.concatenate([vo_merge.query('revenue == 0').index , vo_merge.query('revenue > 100').index])

#drop rows with the offending indices
vo_merge.drop(index=zrev_idx, inplace=True)

In [ ]:
#produce new percentile report for revenue
percentile_report(vo_merge[~vo_merge['purchase_ts'].isna()].revenue, [5, 50, 90])

<a id='pp_dev'></a>
<h3>5.8. Device Usage and Distribution</h3>

<span style='font-family:sabon;font-size:13pt;'>
    In this section we will take a look at the different devices users use to engage with the service.
    </span>

In [ ]:
#prepare data for pie chart - aggregate by device
device_pie = visits_data.\
                groupby('device')\
                    .nunique()[['user_id']]\
                        .rename(columns= {'user_id' : 'device_count'}).reset_index()
#plot pie chart
fig = go.Figure( go.Pie(
    values=device_pie['device_count'],
    labels=device_pie['device'],
    title='Ratio of Desktop to Touch Device Use'
))

fig.update_layout(
    font=dict(
        family="Arial Black",
        size=22,
        color="Black"
    )
    )

fig.update_traces(textposition='auto',
              textinfo='percent+value',
              textfont= dict(
                  size = 18,
                  family = 'Arial Black'
              ),
              marker=dict(
                  colors=['#5C377D', '#ffafcc'],
                  line=dict(
                      color='#000000',
                      width=2
                  )
              )
             )

fig.show()

<span style='font-family:sabon;font-size:13pt;'>
    We find that most usage on the platform comes from desktop devices.
    </span>

<a id='stat_sum'></a>
<h2>6. Traffic Source Overall Performance</h2>

<a id='stat_dev_1'></a>
<h3>6.1 Device Usage Across Sources</h3>

<span style='font-family:sabon;font-size:13pt;'>
    we want to find out if some traffic sources are exclusive or more prevalent in one device than the other, and if some user ids appear in both devices.
    </span>

In [ ]:
#pivot visits data to count unique users in each source by device type
source_pivot = visits_data.pivot_table(
    index= 'source_id',
    columns= 'device',
    values = 'user_id',
    aggfunc= 'nunique'
)

#calculate total users for each source
source_pivot['total'] = source_pivot['desktop'] + source_pivot['touch']

#calculate % desktop users out of total
source_pivot['desktop%'] = round(source_pivot['desktop'] / source_pivot['total'], 1)

#calculate % touch users out of total
source_pivot['touch%'] = round(source_pivot['touch'] / source_pivot['total'], 1)

#show results
source_pivot.head(10).style.background_gradient(subset=['total'],cmap='Greens').format({'desktop%' : '{:.1%}', 'touch%' : '{:.1%}'})

In [ ]:
#plot ratio of devices by source
fig = px.bar(
    source_pivot,
    x=source_pivot.index,
    y=['desktop%', 'touch%'],
    title='Ratio of device Usage across traffic Source'.title(),
    color_discrete_sequence= ['#5C377D', '#ffafcc'],
    template='seaborn'
)
#remove grid
fig.update_yaxes(
                showgrid=False,
                title='Ratio%'
)
fig.update_xaxes(
    showgrid=False,
    title='Campaign ID'
)


#change title attributes
fig.update_layout(
    font=dict(
        family="Arial Black",
        size=14
    ),
    title_x = 0.5,
    xaxis=dict(
        showgrid=False,
        title= dict(
        text = 'Source ID',
        font = subtitle_style
        ),
        tickmode = 'array',
        tickvals = source_pivot.index,
        type = "category"
    )
)
fig.show()

<span style='font-family:sabon;font-size:13pt;'>
    We find that the distribution of desktop usage vs touch device usage is similar across the different channels, with Source 3 being slightly higher on the desktop end.
    </span>

<a id='stat_cost'></a>
<h3>6.2. Average Monthly Costs by Source</h3>

In [ ]:
#add month column for aggregation
costs_data['month'] = pd.to_datetime(costs_data['date'].dt.strftime("%Y-%m"))

#aggregate costs average by month for each source
costs_agg = costs_data\
                .groupby(['source_id','month'], as_index=False)\
                .agg({'costs' : 'mean'})\
                .rename(columns= {'costs' : 'avg_costs'})

#set source_id type to category to comply with some plotly categorization
costs_agg['source_id'] = costs_agg['source_id'].astype('category')

costs_agg.head()

In [ ]:
#plot average monthly costs by source
fig= px.line(
    costs_agg,
    x='month',
    y='avg_costs',
    color='source_id',
    title= f'Average Costs by Month per Traffic Source',
    template='seaborn'

)

#remove grid
fig.update_yaxes(
    showgrid=False,
    title='Cost Average'
)
fig.update_xaxes(
    showgrid=False,
    title='month'
)


#change title attributes
fig.update_layout(
    font=dict(
        family="Arial Black",
        size=12
        ),
    title_x = 0.5
)
fig.show()

<span style='font-family:sabon;font-size:13pt;'>
    Source 3 is clearly the most costly of the sources.<br>
    Later on we will be checking if this cost is justified.
    </span>

<a id='gen_sum'></a>
<h3>6.3. General Statistical Summary by Source</h3>

In [ ]:
#aggregate revenue and user count by source
vo_summary = vo_merge.groupby('source_id', as_index=False).agg({
    'revenue' : ['mean', 'median', 'sum'],
    'user_id' : 'nunique'
})

#rename columns
vo_summary.columns = ['source_id', 'mean_rev', 'median_rev', 'total_rev', 'uniq_users']

#calculate %revenue relative contribution of each source
vo_summary['%rev'] = round(vo_summary['total_rev'] / vo_summary['total_rev'].sum(), 2)

#calculate ARPU for each source
vo_summary['ARPU'] = round(vo_summary['total_rev'] / vo_summary['uniq_users'], 1)

#show sample
vo_summary.head(2)

In [ ]:
#calculate metrics for sessions with purchases
vo_p_summary = vo_merge[~vo_merge['purchase_ts'].isna()].groupby('source_id', as_index=False).agg({
    'user_id' : 'nunique'
}).rename(columns= {'user_id' : 'uniq_active_users'})

#add active user count to summary
vo_summary = vo_summary.merge(vo_p_summary, on='source_id', how='left').fillna(0)

#add ARPPU to summary
vo_summary['ARPPU'] = round(vo_summary['total_rev'] / vo_summary['uniq_active_users'], 1)

#add average daily costs to summary
avg_costs = costs_data\
                .groupby('source_id', as_index=False)\
                .agg({'costs' : 'mean'})\
                .rename(columns = {'costs' : 'mean_cost'})

#merge average daily costs to summary
vo_summary = vo_summary.merge(avg_costs, on='source_id', how='left').fillna(0)

#calculate Conversion Rate to purchasing customers
vo_summary['buyer_conversion'] = round(vo_summary['uniq_active_users'] / vo_summary['uniq_users'], 2)

#show sample of data
vo_summary.head(10)

In [ ]:
#change source_id type to category to be compatible with plotly 
vo_summary['source_id'] = vo_summary['source_id'].astype('category')

color_ticks= vo_summary['%rev']
#bar chart with CR% as Y, %rev as color scale, cost as line, ARPU as indicator
fig = make_subplots(specs=[[{"secondary_y": True}]])

#plot bar chart
fig = go.Figure(
    go.Bar(
        x=vo_summary['source_id'],
        y=vo_summary['buyer_conversion'],
        marker= dict(
            color=vo_summary['%rev'],
            colorscale='brbg',
            showscale=True,
            colorbar= dict(orientation = 'h',
                           tickformat= '.1%',
                          title= dict(
                              text = '%Rev Share',
                              font= dict(
                                  family = 'Arial Black',
                                  size = 15
                                  )   
                              )
                        )
        ),
        name="Conversion Rate",
        text= vo_summary['%rev'],
        texttemplate= '<b style="font-size: 13px">CR: %{y:.2%} <br><br><b style="font-size: 13px">Rev Share: %{text:.2%}</b>'
    )
)

fig.add_trace(
    go.Scatter(
        x=vo_summary['source_id'],
        y=vo_summary['mean_cost'],
        name="Average CAC",
        yaxis="y2"
    )
)

fig.update_layout(
    xaxis=dict(
        showgrid=False,
        title= dict(
        text = 'Source ID',
        font = subtitle_style
        ),
        tickmode = 'array',
        tickvals = vo_summary['source_id'].unique(),
        type = "category"
    ),
    yaxis=dict(
        showgrid=False,
        title= dict(
            text = 'Conversion Rate%',
            font = subtitle_style
        ),
        tickfont=dict(
            family='Arial Black'
        )
    ),
    yaxis2=dict(
        showgrid=False,
        title= dict(
            text = 'Average CAC',
            font = subtitle_style
        ),
        tickfont=dict(
            color="#FF3800",
            family='Arial Black'
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.5
            ),
    title= dict(
               text='Buyer CR%, CAC and %Revenue of each source'.title(),
               font_family='Arial Black',
               font_size=20,
               x=0.5
    )
)

fig.update_annotations(
    font_size=25
)

fig.show()

<span style='font-family:sabon;font-size:13pt;'>
    <b><u>With these preliminary results we see several things:</u></b><br>
    <ul>
        <li>Source 1 Has a very low average CAC and has the best conversion of paying users, however the relative revenue income from this source is only around 20%.</li>
        <li>Source 3 Has the highest average CAC of all sources, a low conversion of paying users and a decent relative revenue income at 24%.</li>
        <li>Source 4 has medium average CAC with the highest relative revenue income.</li>
        <li>Source 5 has medium average CAC with high relative income.</li>
        <li>Source 10 shows poor performance in revenue at 2%, but it is the cheapest source to run, while providing 13% buyer conversion.</li>
    </ul>
    </span>

<a id='marketing'></a>
<h2>7. Marketing Success Analysis</h2>

<span style='font-family:sabon;font-size:13pt;'>
    In this section we will analyze each source's success based on its profitability, conversion efficiency and stability.<br>
    We will also compare the different sources in terms of which ones are better at obtaining new users each month and overall.<br><br>
    For the analysis of conversion and retention we will divide our users into monthly cohorts.<br>
    </span>

<a id='marketing_attr'></a>
<h3>7.1. Attribution</h3>

<span style='font-family:sabon;font-size:13pt;'>
    Before we can analyze our marketing data, we must decide on an <b>attribution model</b> for each user and their main traffic source which is assumed to be responsible for the acquisition of the user and their continued activity on the service.<br>
        In this analysis we'll use a <b>First Touch</b> model and determine that the main source of each user is the one that they used in their first interaction with the service.
    </span>


In [ ]:
#collect the indices of each users first interaction and use it to filter the data
first_dates_atr = vo_merge.loc[
                                #group by user_id to retrieve each users first start_ts index
                                vo_merge.groupby('user_id')['start_ts'].idxmin()
                                ]\
                            .rename(columns= {'source_id' : 'main_source'})

#merge main_source to the main dataset
vo_attr = vo_merge.merge(first_dates_atr[['user_id', 'main_source']], on='user_id')

#create a simple date column
vo_attr['date'] = pd.to_datetime(vo_attr['start_ts'].dt.date)

#print sanity check
print('records before manipulation: {}'.format(vo_merge.shape[0]))
print('records after manipulation: {}'.format(vo_attr.shape[0]))

print('sample of results:')
#show sample of results
vo_attr.sample()

<a id='marketing_cohort'></a>
<h3>7.2. Cohort Prepping</h3>

<span style='font-family:sabon;font-size:13pt;'>
    We will create monthly cohorts of our user base so that we can track retention and growth rate on a monthly basis.
    </span>

In [ ]:
#use our cohort_prep function to create monthly cohorts
session_cohorts = cohort_prep(vo_attr, 'user_id', 'date', 'session', cohort_month= True)

In [ ]:
#aggregate cohorts based on first session month for each source
cohorts_grp = session_cohorts\
                .groupby(['first_session_month', 'main_source', 'lifetime_month'], as_index=False)\
                .agg({'user_id' : 'nunique'})

#define new column names
cohorts_grp.columns = ['first_session_month', 'main_source', 'lifetime_month', 'uq_user_count']

#show sample of results
cohorts_grp.head()

In [ ]:
#for each cohort in each source, calculate conversion rate
cohorts_grp['retention'] = cohorts_grp\
                            .groupby(['first_session_month','main_source'])['uq_user_count']\
                            .transform(
                            #divide each rows unique user count by the max count for each group
                                lambda grp: [round(float(row_x/grp.max()), 4) for row_x in grp]
                            )

#calculate growth rate for each cohort+source
cohorts_grp['retention_change%'] = cohorts_grp\
                                    .groupby(['first_session_month','main_source'])['retention']\
                                    .transform(
                                    #for each cohort+main source we calculate the month over month growth rate
                                        lambda grp: [
                                            round(
                                                float(
                                                #we use enumerate and list comprehension to access the relevant indices
                                                #for the growth rate calculation
                                                (val_x - grp.reset_index(drop=True).loc[idx-1]) / grp.reset_index(drop=True).loc[idx-1]
                                                ), 4) if idx != 0 else np.nan for idx, val_x in enumerate(grp)
                                        ]
                                    )
#show resulting columns
cohorts_grp.head()

<a id='marketing_cohort_res'></a>
<h4>7.2.1. Cohort Initial Results</h4>

So far we have calculated Retention and Stability metrics for each cohort.<br>
We will now present a sample of the results for the first source.<br><br>
The full results will be presented at the end of the next section, where we will include the <b>return on investment</b> for each source and the <b>new customer pull strength</b>

In [ ]:
#show sample of the first source 
for src in cohorts_grp.main_source.unique()[:1]:
    for metric in ['retention', 'retention_change%']:
        if metric == 'retention':
            cohort_pivot = cohorts_grp.query('main_source == @src').pivot_table(index = 'first_session_month', columns= 'lifetime_month', values='retention')
            fig, ax = plt.subplots(figsize=(25,25))
            sns.heatmap(
                data= cohort_pivot,
                annot=True,
                square=True,
                fmt= '.1%',
                #cbar_kws={"shrink": 0.75},
                annot_kws={"fontsize":20},
                linewidths=1,
                linecolor='grey',
                vmax=0.14,
                cmap='YlGn').set_title(f'Source {src} Retention Rate', fontsize=28)

            #change tick labels font size
            cax = plt.gcf().axes[0]
            cax.tick_params(labelsize=18)

            #set axes label
            plt.xlabel("Lifetime (Months)", fontsize = 24)
            plt.ylabel("Cohort Month", fontsize = 24)

            plt.show()
        else:
            cohort_pivot = cohorts_grp.query('main_source == @src').pivot_table(index = 'first_session_month', columns= 'lifetime_month', values='retention_change%')
            fig, ax = plt.subplots(figsize=(25,25))
            sns.heatmap(
                data= cohort_pivot,
                annot=True,
                square=True,
                fmt= '.1%',
                #cbar_kws={"shrink": 0.75},
                linewidths=1,
                annot_kws={"fontsize":20},
                linecolor='grey',
                center= 0,
                vmin = -0.7,
                vmax= 0.5,
                cmap='RdYlGn').set_title(f'Source {src} Retention Change', fontsize=28)

            #change tick labels font size
            cax = plt.gcf().axes[0]
            cax.tick_params(labelsize=18)

            #set axes label
            plt.xlabel("Lifetime (Months)", fontsize = 24)
            plt.ylabel("Cohort Month", fontsize = 24)

            plt.show()

<span style='font-family:sabon;font-size:13pt;'>
    <b style='font-size:18py;'>Monthly Cohorts Retention Results:</b><br><br>
    We see that this source has a good average retention until march for most cohorts.<br>
    This source performed rather well for the first three months in the data, showing an average of 9% retention across the first three cohorts.
    </span>

<a id='marketing_romi'></a>
<h3>7.3. Calculating Return On Marketing Investment</h3>

<span style='font-family:sabon;font-size:13pt;'>
    Here we will calculate how profitable each source is.<br>
    In order to do this we use the formula:<br>
    ${ROMI\%} = \frac{\sum Revenue - \sum Costs}{\sum Costs} $<br><br>
    We will apply this both by month for each source and by total for each source.
    </span>

In [ ]:
#define dataframe for revenue aggregation (filter out non-purchases)
rev_data = vo_attr[~vo_attr['purchase_ts'].isna()][['source_id','purchase_ts','revenue']]

#define column purchase_month
rev_data['purchase_month'] = pd.to_datetime(rev_data['purchase_ts'].dt.strftime('%Y-%m'))

#drop purchase_ts
rev_data.drop('purchase_ts', axis=1, inplace=True)

#update dataframe with aggregated form
rev_data = rev_data.groupby(['source_id','purchase_month'], as_index=False).agg({'revenue' : 'sum'})

#show sample
rev_data.sample()

In [ ]:
#define costs aggregation data as grouped costs by source each month
costs_agg = costs_data.groupby(['source_id','month'], as_index=False).agg({'costs' : 'sum'})

#show sample
costs_agg.sample()

In [ ]:
#outer merge revenue and costs data
romi_agg = rev_data.merge(
    costs_agg, 
    right_on= ['source_id', 'month'],
    left_on=['source_id','purchase_month'],
    how='outer'
    ).drop('purchase_month', axis=1)

#calculate return on marketing investment
romi_agg['%ROMI'] = (romi_agg['revenue'] - romi_agg['costs']) / romi_agg['costs']

#show sample of results
romi_agg.head()

In [ ]:
#plot ROMI% by source each month
fig = px.line(
    romi_agg,
    x=romi_agg['month'],
    y='%ROMI',
    color='source_id',
    template = 'seaborn',
    title= 'ROMI% by Traffic Source',
    width=800,
    height=600
)

#add dash line at 0
fig.add_hline(y=0, line = dict(color = 'black', width = 2, dash = 'dash'))

#update layout
fig.update_layout(
    title= dict(
        font= dict(
            family='Arial Black',
            size=20
        )
    ),
    xaxis= dict(
        title= dict(font = subtitle_style),
        showgrid= False
    ),
    yaxis= dict(
        title= dict(font = subtitle_style),
        tickformat = '.1%',
        tickfont = dict(
            family= 'Arial Black',
            size=15
        ),
        showgrid= False
    )
)

fig.show()

<span style='font-family:sabon;font-size:13pt;'>
    The results show us that only Source 1 is actually profitable in terms of return on investment.<br>
    However ROI is merely one metric that shows us the success of a traffic channel. In marketing, it is common to invest in unprofitable channels if they produce a strong pull of new users, which pay off in the long run.<br>
    In order to determine the pull strength of each source we observe the new users it pulls in every month.
    </span>

<a id='marketing_acq'></a>
<h3>7.4. Acquisition Pull</h3>

<span style='font-family:sabon;font-size:13pt;'>
    In this work we define Acquisition Pull as a standardized measure of the relative power of each source to attract new users.<br>
    we will create this measure by using a min-max normalization method on the count of new users each source was able to acquire each month.
    </span>

In [ ]:
#we'll use the session_cohorts df which is all set with first sesssion dates
source_users_monthly = session_cohorts\
                        .groupby(['main_source', 'first_session_month'], as_index=False)\
                        .agg({'user_id' : 'nunique'})\
                        .rename(columns= {'user_id' : 'new_users'})
source_users_monthly.head()

In [ ]:
#merge the count of new users to romi_agg df
romi_agg = romi_agg.merge(
    source_users_monthly,
    left_on=['source_id','month'],
    right_on=['main_source','first_session_month'], 
    how='left'
        ).drop(['first_session_month','main_source'], axis=1)

#calculate month over month growth rate of new users for each source
romi_agg['new_user_growth'] = romi_agg.groupby(['source_id'])['new_users'].transform(
                                        #for each cohort+main source we calculate the month over month change
                                            lambda grp: [
                                                round(
                                                    float(
                                                    #we use enumerate and list comprehension to access the relevant indices
                                                    #for the change rate calculation
                                                    (val_x - grp.reset_index(drop=True).loc[idx-1]) / grp.reset_index(drop=True).loc[idx-1]
                                                    ), 4) if idx != 0 else np.nan for idx, val_x in enumerate(grp)
                                            ]
                                        )

#calculate acquisition pull
romi_agg['acquisition_pull'] = romi_agg.groupby(['month'])['new_users'].transform(
    lambda grp: round((grp - np.min(grp)) / (np.max(grp) - np.min(grp)),3)
    )

#show sample
romi_agg.head()

<a id='marketing_sum'></a>
<h3>7.5. Marketing Success Summary Visualization</h3>

In [ ]:
#determine y_min and y_max ranges for all plots
min_max = [romi_agg[x].min() for x in ['%ROMI', 'new_user_growth']] + [romi_agg[x].max() for x in ['%ROMI', 'new_user_growth']]
y_min = min(min_max) - 0.1
y_max = max(min_max) + 0.1 

#normalized aquisition pull based on average new users in each source
user_avgs = [
    romi_agg.query('source_id == @src')['new_users'].mean() for src in romi_agg.source_id.unique()
]

normal_nua = (user_avgs - np.min(user_avgs)) / (np.max(user_avgs) - np.min(user_avgs))

#create nua dictionary
nua_source = dict(zip(romi_agg.source_id.unique(), normal_nua))

In [ ]:
#define plot function for source summary
def plot_summary(source):
    for metric in ['retention', 'retention_change%']:
        if metric == 'retention':
            cohort_pivot = cohorts_grp.query('main_source == @source').pivot_table(index = 'first_session_month', columns= 'lifetime_month', values='retention')
            fig, ax = plt.subplots(figsize=(25,25))
            sns.heatmap(
                data= cohort_pivot,
                annot=True,
                square=True,
                fmt= '.1%',
                #cbar_kws={"shrink": 0.75},
                annot_kws={"fontsize":20},
                linewidths=1,
                linecolor='grey',
                vmax=0.14,
                cmap='YlGn').set_title(f'Source {source} Retention Rate', fontsize=28)

            #change tick labels font size
            cax = plt.gcf().axes[0]
            cax.tick_params(labelsize=18)

            #set axes label
            plt.xlabel("Lifetime (Months)", fontsize = 24)
            plt.ylabel("Cohort Month", fontsize = 24)

            plt.show()
        else:
            cohort_pivot = cohorts_grp.query('main_source == @source').pivot_table(index = 'first_session_month', columns= 'lifetime_month', values='retention_change%')
            fig, ax = plt.subplots(figsize=(25,25))
            sns.heatmap(
                data= cohort_pivot,
                annot=True,
                square=True,
                fmt= '.1%',
                #cbar_kws={"shrink": 0.75},
                linewidths=1,
                annot_kws={"fontsize":20},
                linecolor='grey',
                center= 0,
                vmin = -0.7,
                vmax= 0.5,
                cmap='RdYlGn').set_title(f'Source {source} Retention Change', fontsize=28)

            #change tick labels font size
            cax = plt.gcf().axes[0]
            cax.tick_params(labelsize=18)

            #set axes label
            plt.xlabel("Lifetime (Months)", fontsize = 24)
            plt.ylabel("Cohort Month", fontsize = 24)

            plt.show()

    romi_test = romi_agg.query('source_id == @source')

    fig = go.Figure(
        go.Scatter(
            x=romi_test['month'],
            y=romi_test['%ROMI'],
            name="%ROMI"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=romi_test['month'],
            y=romi_test['acquisition_pull'],
            name="Monthly Aquisition Pull"
        )
    )

    fig.add_trace(
        go.Indicator(
            mode = "delta",
            value = romi_test['%ROMI'].mean(),
            delta = {'reference': 0, 'valueformat' : '.1%'},
            title = 'Average %ROMI',
            domain = {'x': [0.05, 0.25], 'y': [0.75, 0.85]}))

    fig.add_trace(
        go.Indicator(
            mode = "delta",
            value = romi_test['new_user_growth'].mean(),
            delta = {'reference': 0, 'valueformat' : '.1%'},
            title = 'Avg User Growth',
            domain = {'x': [0.38, 0.58], 'y': [0.65, 0.95]}))

    fig.add_trace(
        go.Indicator(
            mode = "number",
            value = round(nua_source[source],2),
            title = 'Overall Aquisition Pull',
            domain = {'x': [0.75, 0.90], 'y': [0.75, 0.85]}))

    fig.update_layout(
        yaxis=dict(
            title= dict(
                text = '%ROMI',
                font = subtitle_style
                       ),
            tickfont=dict(
                subtitle_style
            ),
            tickformat = '.1%',
            range=[y_min,y_max],
        ),
        xaxis= dict(
            title= dict(
                text = 'Month',
                font = subtitle_style
                       )
        ),
        title= dict(
            text = f'Source {source}: ROMI vs New User Growth Month over Month',
            font = dict(
                family='Arial Black',
                size= 20
            ),
            x=0.5
                   )
       )
    fig.add_hline(y=0, line = dict(color = 'rgb(100, 10, 100)', width = 2, dash = 'dash'))

    fig.show()

<a id='marketing_s1'></a>
<h4>7.5.1. Analysis of Source 1</h4>

In [ ]:
plot_summary(source=1)

<span style='font-family:sabon;font-size:13pt;'>
    This source has overall good ROMI performance, with 19.9% positive average return on investment (with a slight dip during Jan 2018), a positive 6.4% new user growth month over month average, but a very low user pull strength average of 0.06 compared to other sources.<br>
    This source sports over 5% retention in the first four months for most of the cohorts, with its best performance being throughout the first cohort.<br>
    We see two spikes of an increase in return of users in the July cohort during January 2018, however these results are not necessarily consistent throughout the different cohorts.<br>
    <u><b>Summary:</b></u><br>
    This source has good performance in its profitability and is the only source which is profitable, boasting a 19.9% return on investment, and ending the test period strong with 42% return during May of 2018.<br>
    Although this source is not very good at pulling new users, it's hard to discount its importance due to the strong ROMI and the positive average month over month growth rate.
    </span>

<a id='marketing_s3'></a>
<h4>7.5.2. Analysis of Source 3</h4>

In [ ]:
plot_summary(source=3)

<span style='font-family:sabon;font-size:13pt;'>
    This source has a very stark contrast between profitability and user pulling.<br>
    We see a very low return on investment at a stable average of -79.3%. However its ability to pull users is one of the best among the different channels, sitting at 0.91 with this source being #1 in user pulling during the first 4 months in the testing period.<br>
    Important to note that towards the end of the data period it reaches its lowest point at 0.7 with a possible downwards trend.<br>
    Retention in this source is rather low, averaging 4-5% for the initial cohorts for the first 6 months, however this begins to decline starting with the 2017 september cohort.<br> 
    February 2018 to April 2018 show a very negative retention change across all cohorts, as customers continue to abandon the use of this traffic channel.<br>
    In addition we see a consistent negative growth rate starting from the september 2017 cohort.<br>
    <u><b>Summary:</b></u><br>
    This source has high losses but is very good at pulling new users in.<br>
    Its strongest effects are seen in the first few months and begin to decline in user retention. However, its excellent ability to pull in new users remains effective for 8 months.
    </span>

<a id='marketing_s4'></a>
<h4>7.5.3. Analysis of Source 4</h4>

In [ ]:
plot_summary(source=4)

<span style='font-family:sabon;font-size:13pt;'>
    This source has low profitability sitting at -45.6% ROMI. However its pull strength is the best out of all the sources consistently throughout the test period.<br>
    We also see a good average month over month growth rate of 8%.<br>
    This source has its best retention rates during the first few cohorts for 3-4 months, keeping retention of 5% on average.<br>
    Looking at the user retention change rate this source performs worse during November, February, and April.<br>
    The 2017 June to 2017 August cohorts have mostly positive growth, which begins to decline for cohorts starting 2017 September.<br>
    <u><b>Summary:</b></u><br>
    While this source has a loss at -45.6% ROMI, its ability to pull new users is the best of all the sources and its user month over month loss is relatively low making it a prime candidate for conservation.<br>
    </span>

<a id='marketing_s5'></a>
<h4>7.5.4. Analysis of Source 5</h4>

In [ ]:
plot_summary(source=5)

<span style='font-family:sabon;font-size:13pt;'>
    This source produces a 36.6% loss, with a moderately strong user pull at 0.65 which peaks during July 2017 to September 2017.<br>
    average User growth is positive at 3.8%<br>
    Retention rates for this source are rather low across the board.<br>
    Only the first cohort shows consistent positive user retention growth, which declines significantly for cohorts past september 2017.<br>
    <u><b>Summary:</b></u><br>
    This source has low profitability and a mid range pull strength.<br>
    The positive average user growth might not be strong enough to compensate for the losses.
    </span>

<a id='marketing_s10'></a>
<h4>7.5.5. Analysis of Source 10</h4>

In [ ]:
plot_summary(source=10)

<span style='font-family:sabon;font-size:13pt;'>
    This source shows a low profitability with a -45.9% ROMI<br>
    With a very low user pull strength, the user growth rate becomes less significant, as the fluctuating outliers make it an innaccurate metric for this source.<br>
    This sources retention performs well during the cohorts of August 2017 to December 2018, however the effectiveness reduces after 2-3 months for each one.<br>
    Growth rate is sporadic with several outliers on both ends of the spectrum.<br>
    While this source performs poorly in ROI and user pull, it is very cheap while having 13% conversion to paying users. 
    <br><u><b>Summary:</b></u><br>
    This source has very low profitability, low user pull rate and low retention. However it is cheap and is able to convert 13% users to paying users.
    </span>

<a id='marketing_reppurchases'></a>
<h3>7.6. Channel Purchase Redirection</h3>

<span style='font-family:sabon;font-size:13pt;'>
    We have seen that some channels are good for certain things, such as pulling in new users, converting users to buyers and providing low cost acquisition options.<br>
    Because of this, we are interested to see if a cheap source, such as 10 for example, could be converting users to purchasers who will continue to purchase through other channels, or produce significantly more revenue through other channels.<br>If such a thing occurs consistently, then surely that will make certain channels worth the cost.
    </span>

In [ ]:
#create concurrent purchases tables for each source
for source in sorted(vo_merge.source_id.unique()):
    #pull first purchases of users from source of their first purchase
    users = vo_merge\
                .loc[vo_merge[~vo_merge['purchase_ts'].isna()]\
                .groupby('user_id')['purchase_ts'].idxmin()]\
                .query('source_id == @source')[['user_id','purchase_ts','revenue']]
    
    #filter main dataset for same users while ignoring their first purchase
    users_next_purchase = vo_merge[(vo_merge['user_id'].isin(users['user_id'])) & ~(vo_merge['purchase_ts'].isna()) & ~(vo_merge.index.isin(users.index))]
    
    #aggregate all purchases by source for these users after their first purchase
    purchase_agg = users_next_purchase\
                    .groupby('source_id')\
                    .agg({'user_id' : 'count', 'revenue' : 'mean'})\
                    .rename(columns = {'user_id' : 'purchase_count', 'revenue' : 'avg_revenue'})
    
    #add purchase% column that shows which source gets most purchases from these users
    purchase_agg['%purchases'] = purchase_agg['purchase_count'] / purchase_agg['purchase_count'].sum()
    
    #display results with highlighted column
    print(f'Repeat purchases of first buyers in source {source}')
    display(purchase_agg.style.background_gradient(subset=['%purchases'],cmap='Greens').format({'%purchases' : '{:.1%}'}))

<span style='font-family:sabon;font-size:13pt;'>
    <b>Results:</b><br>
    <ul>
        <li>Source 1 - Most users who first purchased on this source, come back to it 50% of the time, with some redirected to source 5.</li>
        <li>Source 3 - Users who first purchased from this source, distribute their repeat purchases around sources 1, 3 and 4 mostly</li>
        <li>Source 4 - Users who first purchases from this source continue purchasing through it mostly, and some are directed to source 1 and 3</li>
        <li>Source 5 - Most users who first purchased on this source, come back to it 62% of the time, with some continuing with source 1</li>
        <li>Source 10 - Most users who first purchased on this source, come back to it 46% of the time, however the repeat purchase rates are very low for this source.</li>
    </ul>
    </span>

<a id='conc_rec'></a>
<h2>8. Conclusions and Recommendations</h2>

<span style='font-family:sabon;font-size:13pt;'>
    We recieved marketing data consisting of user sessions, purchases and marketing costs.<br>
    Our task was to analyze the different traffic channels and determine which one to continue funding and which one to abandon.<br><br>
    In order to make this decision, we processed the data and merged it, we explored session length, revenue and origin.<br>
    We constructed visualizations using cohorts and using general summary statistics.<br><br>
    <b>Through looking at each traffic source we construct and make the following recommendations for each one:</b><br><br>
    Source 1: This source has consistent large return on investment, and the largest % conversion of users to paying users. <b>Although low on user pull strength, this source boasts low CAC and big returns, and therefore should be allowed to continue to operate.</b><br><br>
    Source 3: This is the most costly source with costs peaking during september to march - times where retention is fairy stable.<br>While being the most costly, it is also one of the strongest puller of new users, with its buyers spreading to other channels as time goes on.<br>
    Interestingly the investment put into this source does not seem to correlate with its ability to draw in users, as we can see the user pull is very high during the months before september, where the cost was significantly lower.<br>
    <b>At its current form, it would be unfeasible to continue operating this source, however perhaps lowering the investment while monitoring the user pulling abilities of this source could prove beneficial.</b><br><br>
    Source 4: This source has the strongest average user pull, with decent chunk of profit loss. However we also see that it produces 28% of the total revenue out of all the sources, with only 10% paying user conversion.<br> Meaning this source attracts high value customers potentially.<br>
    Furthermore, we see that as we increase the costs, we have higher user pull and higher ROMI%.<br>
    Therefore <b>we suggest keeping this source operational, as it brings in valuable customers which are essential for long term growth.</b><br><br>
    Source 5: For this source, July to October seem to be the best time to invest in this channel, showing an increase in ROMI during those times.<br> In general this source is responsible for a big chunk of the revenue stream (26%) and is moderately good at pulling in new users.<br><b>Our recommendation for this one is to keep it at low investment and pump budget into it during July to October for maximum efficiency.</b><br><br>
    Source 10: With large negative ROMI% and weak pull of new users, in addition to low retention, this source appears to be the least profitable and has the least prospects for the future.<br>While this is true, it does show an efficient conversion of users to buyers, and has the cheapest acquisition cost of all the sources.<br>
    <b>Ultimately, though this channel is cheap, it provides low churn of users and those who do end up making purchases do not normally transfer to other sources to make repeat purchases. Therefore we think it is best to discontinue this source.</b>
    <br><br>
    In the next several months, it is imperative to continue and follow the trends of the various metrics shown in this work and act according to the performance of the continued traffic channels.
    </span>